In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math

In [2]:
DEGREES = [3, 4]
PROBABILITIES = [0.5, 0.5]

MIN_DEG = min(DEGREES)
MAX_DEG = max(DEGREES)
LCM = math.lcm(*DEGREES)

In [3]:
def random_walk(n):
    return [np.random.randint(0,LCM-1) for _ in range(n)]

def height_in_constant_degree_tree(d, walk):
    height = 0
    
    for step in walk:
        if int(step * d / LCM) == 0 and height > 0:
            height -= 1
        else:
            height += 1
    
    return height

def min_height(walk):
    return height_in_constant_degree_tree(MIN_DEG, walk)

def max_height(walk):
    return height_in_constant_degree_tree(MAX_DEG, walk)

def walk_goes_higher_than(d, walk):
    return min_height(walk) > d

In [4]:
def UGW(height, degs=DEGREES, probs=PROBABILITIES):
    normalization = sum([ d * p for d, p in zip(degs, probs) ])
    offspring_probs = [ d * p / normalization for d, p in zip(degs, probs) ]
    
    T = nx.Graph()
    T.add_node('')
    
    root_deg = np.random.choice(degs, p=probs)
    current_level = [ str(i) for i in range(root_deg) ]
    T.add_edges_from([ ('', v) for v in current_level ])
    
    current_height = 1
    while current_height <= height:
        next_level = []
        
        for v in current_level:
            v_deg = np.random.choice(degs, p=offspring_probs)
            new_nodes = [ v + str(i) for i in range(1, v_deg)]
            next_level.extend(new_nodes)
            T.add_edges_from([ (v, u) for u in new_nodes ])
        
        current_level = next_level
        current_height += 1
    
    return T

In [5]:
def walk_in_tree(walk, tree):
    current_node = ''
    
    for step in walk:
        reduced_step = int(step * tree.degree[current_node] / LCM)
            
        if reduced_step == 0 and current_node != '':
            next_node = current_node[:-1]
        else:
            next_node = current_node + str(reduced_step)
        
        current_node = next_node
    
    return current_node

In [6]:
def ball(graph, root, radius):
    b = nx.Graph()
    b.add_edges_from(nx.bfs_edges(graph, root, depth_limit=radius))
    b.nodes[root]['root'] = True
    return b

def ball_node_match(n1, n2):
    return n1 == n2

In [7]:
def mappings(xs, ys):
    if len(xs) == 0:
        return [[]]
    else:
        x, *zs = xs
        
        smaller_mappings = mappings(zs, ys)
        return [
            [(x,y)] + sm
            for y in ys
            for sm in smaller_mappings
        ]

In [8]:
def possible_balls(radius, degs=DEGREES):
    if radius == 1:
        balls = []
        
        for d in degs:
            b = nx.Graph()
            b.add_edges_from([('', str(i)) for i in range(d)])
            b.nodes['']['root'] = True
            balls.append(b)
            
        return balls
    
    else:
        smaller_balls = possible_balls(radius-1, degs)
        balls = []
        
        for sb in smaller_balls:
            leaves = [node for node in sb if len(node) == radius-1]
            
            for m in mappings(leaves, degs):
                b = sb.copy()
                
                for l, d in m:
                    b.add_edges_from([(l, l+str(i)) for i in range(1,d)])
                
                if all([not nx.is_isomorphic(b, x, node_match=ball_node_match) for x in balls]):
                    balls.append(b)

        return balls

In [13]:
num_samples = 5
num_subsamples = 10
ball_radius = 2
walk_length = 7

all_balls = possible_balls(ball_radius)
num_balls = len(all_balls)

samples = []

for _ in range(num_samples):
    w = random_walk(walk_length)
    while not walk_goes_higher_than(2 * ball_radius, w):
        w = random_walk(walk_length)
        
    ugw_height = max_height(w) + ball_radius
        
    statistic = np.zeros((num_balls, num_balls))
        
    for __ in range(num_subsamples):
        t = UGW(ugw_height)
        
        start = ''
        end = walk_in_tree(w, t)
        
        bstart = ball(t, start, ball_radius)
        bend = ball(t, end, ball_radius)
        
        for i, b1 in enumerate(all_balls):
            if nx.is_isomorphic(bstart, b1, node_match = ball_node_match):
                for j, b2 in enumerate(all_balls):
                    if nx.is_isomorphic(bend, b2, node_match = ball_node_match):
                        statistic[i,j] += 1
                        break
                break
        
    if int(statistic.sum()) != num_subsamples:
        raise Exception("at least one of the sampled balls did not match!")
    
    samples.append(statistic)

samples

[array([[0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 2., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0

In [ ]:
i = 0
for x in possible_balls([3,4],2):
    plt.figure(i)
    i += 1
    nx.draw(x, with_labels=True)

In [ ]:
len(mappings(range(3),range(4)))

In [ ]:
t1 = nx.Graph()
t1.add_edges_from([(1,2), (2,3)])
t1.nodes[1]['root'] = True

t2 = nx.Graph()
t2.add_edges_from([('a','b'), ('b','c')])
t2.nodes['c']['root'] = False

nx.is_isomorphic(t1,t2, node_match=ball_node_match)

In [ ]:
w = random_walk(3)
w

In [ ]:
walk_in_tree(w, t)

In [ ]:
t = UGW([3,4],[0.5,0.5],2)
nx.draw(t, with_labels=True)

In [ ]:
t.nodes['311']['root'] = True
t.nodes['311']

In [ ]:
g = ball(t,'311',3)
nx.draw(g, with_labels=True)
g.nodes['311']

In [ ]:
for node in g:
    print(node)

In [ ]:
a = { 'root' : True}
b = { 'root' : True}

In [ ]:
a == b

In [ ]:
c = {}
d = {}

In [ ]:
a == c

In [ ]:
for a in range(10):
    if a % 2 == 0:
        print(a)

In [ ]:
a = [1,2]
a.append(3)
a

In [ ]:
[1] + [2]

In [ ]:
a = np.zeros((2,2))

In [ ]:
a

In [ ]:
a[1,1] = 1

In [ ]:
a

In [ ]:
a[1,1] += 1

In [ ]:
a

In [ ]:
np.sum(a, (0))

In [ ]:
a.sum()

In [ ]:
raise Exception("stuff")